```java
public static void printClassLoaders() {
        System.out.println("Classloader of current class:"
                + new Main().getClass().getClassLoader());

        System.out.println("Classloader of ArrayList:"
                + ArrayList.class.getClassLoader());
    }

//Classloader of current class:jdk.internal.loader.ClassLoaders AppClassLoader@251a69d7
//Classloader of ArrayList:null
```
在此可以看到两个classloader: application, and bootstrap (displayed as null). 其实还有一个extension classloader. 主要就这三个classloader. 

需要知道字节码文件是动态导入的, 当导入A类的时候里面代码引用或继承了B类, 那JVM也会请求导入B类(写代码的时候一般一个类为一个源文件`.java`, 一个源文件就是一个编译单元, 经过javac编译器编译成bytecode). 

---

JVM 包括两部分 Classloader, Execution Engine, JVM开始工作的第一阶段就是通过classloader**把类(bytecode)导入到内存ram中**, 程序运行不就得先从磁盘进入内存吗. 每个java类或者接口一般都是单独在一个`.java`文件, 所以JVM会导入多个`.class`文件, 但有个主文件, 就是那个含有`static main()`, 也就是说classloader会先load含有这个静态方法的字节码文件. 之后再导入其他类(**loading的过程是dynamic**, 用到谁导入谁, 不是一下全部导入), 如下:

All the subsequent class loading attempts are done according to the class references in the already-running classes as mentioned in the following cases:
- When bytecode make a static reference to a class (e.g. System.out)
- When bytecode create a class object (e.g. Person person = new Person("John"))


# What is ClassLoader in Java
---

Classloader 也是类你可以在代码中调用. 

**ClassLoader in Java is a class** that is used to load class files in Java. Java code is compiled into a class file by `javac` compiler and JVM executes the Java program, by executing byte codes written in the **class file**. ClassLoader is responsible for loading class files from file systems, networks, or any other source. 

There is three default class loader used in Java, Bootstrap, Extension, and System or Application class loader. 

> Every class loader has a **predefined location**, from where they load class files. The bootstrap class loader is responsible for loading standard JDK class files from `rt.jar` and it is the parent of all class loaders in Java. 

1) Bootstrap ClassLoader - JRE/lib/rt.jar

2) Extension ClassLoader - JRE/lib/ext or any directory denoted by java.ext.dirs

3) Application ClassLoader - CLASSPATH environment variable, -classpath or -cp option, Class-Path attribute of Manifest inside JAR file.

The **bootstrap class loader** doesn't have any parents if you call `String.class.getClassLoader()` it will return `null` and any code based on that may throw `NullPointerException` in Java.

**Extension class loader** delegates class loading request to its parent, Bootstrap, and if unsuccessful, loads class form `jre/lib/ext` directory or any other directory pointed by `java.ext.dirs` system property. Extension ClassLoader in JVM is implemented by sun.misc.Launcher$ExtClassLoader. 

**Application class loader** is a child of Extension ClassLoader. Also, except for the Bootstrap class loader, which is implemented in the native language mostly in C,  all  Java class loaders are implemented using java.lang.ClassLoader.

https://javarevisited.blogspot.com/2012/12/how-classloader-works-in-java.html

# How Do Class Loaders Work?
---

还记得前面我们说JVM是动态导入类吗? 没错, 每次用到某个类, JVM就会请求导入, 然后一些java内置方法(`java.lang.ClassLoader.loadClass()`)被调用开始搜索那个类, 如果找不到就会抛出异常. 接下来我们来看一下`loadClass()`都做了什么:

When JVM request for a class, it invokes a `loadClass()` method of the `java.lang.ClassLoader` class by passing the fully classified name of the class. The loadClass() method calls for `findLoadedClass()` method to check that the class has been already loaded or not. It is required to avoid loading the class multiple times.

If the class isn't already loaded, it delegates the request to the parent class loader. This process happens recursively. Eventually, if the parent class loader doesn’t find the class, then the child class will call the `java.net.URLClassLoader.findClass()` method to look for classes in the file system itself. 

If the last child class loader isn't able to load the class either, it throws `java.lang.NoClassDefFoundError` or `java.lang.ClassNotFoundException`.

这里可能会对 parent class loader有疑问, 导入类的时候JVM会先请求application loader, 如果要导入的类没有被导入, 那application loader 就会请求它的父类也就是extension loader, 然后extension loader可能会请求它的父类 bootstrap loader, 如下图: 

![Capture-2023-01-08-191701.png](attachment:Capture-2023-01-08-191701.png)

Let's look at an example of the output when `ClassNotFoundException` is thrown:

```java
java.lang.ClassNotFoundException: com.baeldung.classloader.SampleClassLoader    
    at java.net.URLClassLoader.findClass(URLClassLoader.java:381) 
    at java.lang.ClassLoader.loadClass(ClassLoader.java:424)    
    at java.lang.ClassLoader.loadClass(ClassLoader.java:357)    
    at java.lang.Class.forName0(Native Method)    
    at java.lang.Class.forName(Class.java:348)
```

If we go through the sequence of events right from calling `java.lang.Class.forName()`, we can see that it first tries to load the class through the parent class loader, and then `java.net.URLClassLoader.findClass()` to look for the class itself.

When it still doesn't find the class, it throws a `ClassNotFoundException`.

---

三个原则:

**Visibility principle** states that child ClassLoader can see **the class** loaded by the parent ClassLoader, but vice versa is not true. It means if Application ClassLoader loads Demo.class, in such case, trying to load Demo.class explicitly using Extension ClassLoader throws `java.lang.ClassNotFoundException`.

According to **uniqueness principle**, a class loaded by a Parent should not be loaded by Child ClassLoader again. Though it's completely possible to write a class loader that violates Delegation and Uniqueness principles and loads class by itself, it's not something which is beneficial. You should follow all class loader principles while writing your own ClassLoader.

Delegation principles, ...

# 总结
---

class loading发生在两个时间段, 第一个是最初的时候JVM导入主类, 第二个是之后动态的导入类, JVM动态请求导入. 这也是JVM工作的主要逻辑. 然后JVM有两个部件, Classloader, Execution Engine(Interpreter, JIT compiler and Garbage collector). 
